In [1]:
import re
import numpy as np
import pandas as pd
from unidecode import unidecode

In [2]:
# 从txt中获取数据并查看长度
types = []
for line in open('instance_types_en.txt', "r",encoding='UTF-8'):
    types.append(line)
len(types)

5214242

In [3]:
types = types[1:5214240]

In [4]:
relationship5 = []
for j in types:
    j = unidecode(j)
    # 找到所有网址，删除最后一个
    last = re.findall('<http://.*?> ',j)
    j = j.replace(last[-1],'')
    result1 = re.search('/[^/]*>',j)
    start_name = []
    if result1:
        start_name.append(result1.group(0))
    else:
        continue
    result2 = re.findall('[/|#]+[^/#]+>',j)
    result2 = result2[1:]
    relationship5.append(start_name+result2)

In [5]:
df6 = pd.DataFrame(relationship5, columns = ['start_name','relationship','end_name','others'])

In [6]:
# 删除多余的符号
df6['start_name'] = df6['start_name'].str.replace('[/>#]+', '')
df6['start_name'] = df6['start_name'].str.replace('_+', ' ')

df6['end_name'] = df6['end_name'].str.replace('[/>#]+', '')

df6['relationship'] = df6['relationship'].str.replace('[/>#]+', '')

print(len(df6))
df6.head()

5214239


,start_name,relationship,end_name,others
0,Achilles,type,Thing,None
1,An American in Paris,type,Thing,None
2,Actrius,type,Film,None
3,Animalia (book),type,Book,None
4,Agricultural science,type,Thing,None


In [7]:
df6 = df6.drop('others', axis = 1)
df6 = df6.drop('relationship', axis = 1)
df6.head()

,start_name,end_name
0,Achilles,Thing
1,An American in Paris,Thing
2,Actrius,Film
3,Animalia (book),Book
4,Agricultural science,Thing


In [8]:
lab_dict = df6.set_index('start_name')['end_name'].to_dict()

In [9]:
lab_dict

{'Achilles': 'Thing',
 'An American in Paris': 'Thing',
 'Actrius': 'Film',
 'Animalia (book)': 'Book',
 'Agricultural science': 'Thing',
 'Alain Connes': 'Scientist',
 'Allan Dwan': 'Person',
 'Allan Dwan 1': 'PersonFunction',
 'Allan Dwan 2': 'PersonFunction',
 'Allan Dwan 3': 'PersonFunction',
 'Astronomer': 'Thing',
 'Academy Award for Best Production Design': 'Award',
 'ASCII': 'Thing',
 'Alchemy': 'Thing',
 'Aristotle': 'Philosopher',
 'Austroasiatic languages': 'Thing',
 'Afroasiatic languages': 'Thing',
 'Anarchism': 'Thing',
 'Altruism': 'Thing',
 'Apollo': 'Thing',
 'Animation': 'Thing',
 'Academy Awards': 'Award',
 'Ada': 'Thing',
 'Alabama': 'AdministrativeRegion',
 'American National Standards Institute': 'Organisation',
 'Demographics of Angola': 'Thing',
 'Anthropology': 'Thing',
 'Abraham Lincoln': 'OfficeHolder',
 'Abraham Lincoln 1': 'TimePeriod',
 'Abraham Lincoln 2': 'TimePeriod',
 'Abraham Lincoln 3': 'TimePeriod',
 'Animal Farm': 'Book',
 'Actinopterygii': 'Thing'

In [10]:
entity = pd.read_csv('entity.csv',encoding = 'utf-8')
entity

,Unnamed: 0,entityName,entityId:ID,entityProperty
0,0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb..."
1,1,New Road Team,1,"name: New Road, name: New Road Team, nick: N..."
2,2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0..."
3,3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ..."
4,4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa..."
...,...,...,...,...
4691096,5822928,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo..."
4691097,5822929,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind..."
4691098,5822930,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi..."
4691099,5822931,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa..."


In [11]:
entity = entity.drop('Unnamed: 0', axis = 1)
entity

In [15]:
entity[':LABEL'] = entity['entityName']
entity[':LABEL'] = entity[':LABEL'].map(lab_dict)

In [16]:
entity

,entityName,entityId:ID,entityProperty,:LABEL
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person
...,...,...,...,...
4691096,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship
4691097,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person
4691098,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person
4691099,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation


In [19]:
entity = entity[entity[':LABEL'].notnull()]
entity

,entityName,entityId:ID,entityProperty,:LABEL
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person
...,...,...,...,...
4691096,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship
4691097,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person
4691098,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person
4691099,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation


In [23]:
def find_all_index(arr,item):
    return [i for i,a in enumerate(arr) if a==item]
find_all_index(entity['entityName'],None)

[]

In [24]:
len(set(entity[':LABEL']))

401

In [25]:
entity['entityProperty'] = entity['entityProperty'].str.replace('@','')

In [26]:
entity['entityName'] = entity['entityName'].str.replace('@','')

In [27]:
entity['entityProperty'] = entity['entityProperty'].str.replace('\\','')

In [28]:
entity.to_csv('entity.csv')